In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor

In [32]:
import os
cwd = os.getcwd()
project_path = (cwd, None)
while project_path[1] != "Code":
    project_path = os.path.split(project_path[0])
project_path = project_path[0]

In [33]:
df = pd.read_csv(project_path + "\\Datasets\\data_in.csv")
train=df.sample(frac=0.8)
test=df.drop(train.index)

y_train = train[["FARE"]]
y_test = test[["FARE"]]

In [34]:
X_train_dist = train[["Distance"]]
X_test_dist = test[["Distance"]]

X_train_simd = train[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "Distance", "FARE"}))]
X_test_simd = test[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "Distance", "FARE"}))]

X_train_full = train[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "FARE"}))]
X_test_full = test[list(set(df.columns).difference({"ORIGIN_CODE", "DESTINATION_CODE", "FARE"}))]

In [35]:
rfr_dist = RandomForestRegressor()
rfr_dist.fit(X_train_dist, y_train)
print(rfr_dist.score(X_test_dist, y_test))

C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8876834406466576


C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\sklearn\ensemble\base.py:157: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [36]:
rfr_simd = RandomForestRegressor()
rfr_simd.fit(X_train_simd, y_train)
print(rfr_simd.score(X_test_simd, y_test))

C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8683191293957375


C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\sklearn\ensemble\base.py:157: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [37]:
rfr_full = RandomForestRegressor()
rfr_full.fit(X_train_full, y_train)
print(rfr_full.score(X_test_full, y_test))

C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.9619809706016128


C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\sklearn\ensemble\base.py:157: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [20]:
for max_features in ("auto", "sqrt", "log2"):
    for criterion in ("mae", "mse", "poisson"):
        rfr = RandomForestRegressor(max_features = max_features, criterion = criterion, n_estimators = 10)
        rfr.fit(X_train_full, y_train)
        print(f"max_features: {max_features}, criterion: {criterion} :: {rfr_full.score(X_test_full, y_test)}")

C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


KeyboardInterrupt: 